In [4]:
import os 
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
BASE = "RAG/DATA/CLEANED"
from dotenv import load_dotenv
load_dotenv()
import psycopg2
import requests

In [5]:
def generate_embedding(text:str):
    url = "https://lamhieu-lightweight-embeddings.hf.space/v1/embeddings"
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    data = {
        "model": "snowflake-arctic-embed-l-v2.0",
        "input": text
    }

    response = requests.post(url, headers=headers, json=data)
    if response.ok:
        return response.json()["data"][0]["embedding"]
    else:
        raise Exception("No response")

In [6]:
def create_connection():
    connection = psycopg2.connect(
        host=os.getenv("db_host", "localhost"),
        port=int(os.getenv("db_port", "5432")),
        dbname=os.getenv("dbname", "pgvector"),
        user=os.getenv("db_user", "postgres"),
        password=os.getenv("db_password", "postres")
        )
    return connection , connection.cursor()

In [7]:
connection , cursor = create_connection()

In [8]:
cursor.close()
connection.close()

In [9]:
def check_table_exists(table_name):
    table_name = table_name.strip().lower()
    cursor.execute("""
        SELECT EXISTS (
            SELECT 1
            FROM information_schema.tables 
            WHERE table_name = %s
        );""", (table_name,))    
    return cursor.fetchone()[0]

In [10]:
def create_table(table_name,*,vector_size=1024):
    table_name = table_name.strip().lower()
    cursor.execute(
    f"""
        CREATE TABLE {table_name} (
            id bigint primary key generated always as identity,
            chunk text,
            embedding vector({vector_size})
        ) WITH (OIDS=FALSE);
    """
    )
    connection.commit()

def drop_table(table_name):
    table_name = table_name.strip().lower()
    cursor.execute(f"drop table {table_name};")
    connection.commit()

In [11]:
def insert_text_data(table_name , chunk):
    embedding = generate_embedding(chunk)
    embedding_str = embedding.__repr__().lower()
    cursor.execute(
    f"""
        INSERT INTO {table_name}(chunk, embedding) 
        VALUES ('{chunk}', '{embedding_str}');
    """)
    connection.commit()

In [9]:
from langchain_community.document_loaders import TextLoader
import re
text_files = list(os.listdir("./RAG/DATA/CLEANED/"))
text_file = text_files[0]
bank = "SBI"
loader = TextLoader(
    f"{BASE}/{bank}_output_cleaned.md"
    # "test.txt"
)
documents = loader.load()
documents[0].page_content = re.sub(r'[^a-zA-Z0-9\\s+\-*%&.]' , " " , re.sub("\\n+|\\s+" , " ",documents[0].page_content))    
text_splitter = CharacterTextSplitter(chunk_size=900, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

In [15]:
docs[0].page_content

'**State Bank of India Terms of Service  Terms & Conditions ** **DEFINITIONS** In these Terms of Service  Terms & Conditions   unless the context indicates otherwise  - **The Bank** refers to State Bank of India  established by an Act of Parliament  with its Corporate Office at  State Bank Bhavan   Mumbai. - **OnlineSBI** means the Internet Banking facility offered by the Bank. - **The Site** refers to www.onlinesbi.com. - **Internet Banking Services** include account access  transactions  and other services provided via the Bank s website. - **User** refers to a Bank customer authorized to use Internet Banking Services. - **Confidential Personal Information** includes data obtained by the Bank for service provision. - **Account** refers to eligible accounts  e.g.  Savings  Current  Fixed Deposit  linked to Internet Banking. - **Demat Account** holders must ensure the first holder s name matches the Bank Account. The Bank disclaims liability for data accuracy from NSDL CDSL. - **Future

In [49]:
import re
re.sub(r'[^a-zA-Z0-9\\s+\-*%&.]' , " " , re.sub("\\n+|\\s+" , " " , "-*&^%$#        @!./hel+\n\n\nn            i\nce\n\nto"))

'-*& %     . hel+ n i ce to'

In [12]:
def close_db():
    cursor.close()
    connection.close()
close_db()

In [13]:
connection , cursor = create_connection()



In [55]:
import re
for doc in docs:
    insert_text_data(table_name,re.sub(r'[^a-zA-Z0-9\\s.]', ' ', doc.page_content))

In [ ]:
embedding = generate_embedding("chunk hello world")
embedding_str = '{' + ','.join(map(str, embedding)) + '}'
embedding_str

'{-0.0007462346693500876,0.008901509456336498,-0.027767740190029144,-0.0002123352896887809,-0.000499236979521811,0.027094975113868713,-0.0004941985243931413,-0.033615950495004654,-0.01809828355908394,-0.04523291811347008,0.04478219524025917,-0.02496357634663582,0.03901410102844238,0.00503667863085866,-0.031592413783073425,0.07855739444494247,-0.05959761515259743,-0.015555690042674541,0.0012881729053333402,0.01782648265361786,-0.13933828473091125,0.008183450438082218,0.01422177068889141,-0.002075027208775282,0.09265418350696564,0.009679647162556648,0.07735160738229752,-0.007362628821283579,-0.04454068839550018,-0.0472024604678154,0.044497955590486526,0.04765801131725311,0.017805760726332664,-0.06338197737932205,-0.04586649686098099,-0.015553153119981289,0.004697741940617561,0.011302742175757885,-0.04742808640003204,-0.026448892429471016,-0.0037802511360496283,0.018587682396173477,0.006848130840808153,-0.0033049623016268015,0.00823486689478159,-0.07343783229589462,-0.03959565982222557,0.

In [14]:
def query_relevant_data(table_name , querry_text):
    querry_embedding = generate_embedding(querry_text)
    cursor.execute(
    f"""
        SELECT * FROM {table_name} 
        ORDER BY embedding <=> '{querry_embedding}'
        LIMIT 5;
    """)
    
    return cursor.fetchall()

In [15]:
ans = query_relevant_data("KOTAK" ,"tds deduction")

In [16]:
ans

[(1,
  'Based on the provided text  the **exact sentences or clauses identified as part of the  terms and conditions ** are  1. **Premature Withdrawal **  premature withdrawal  interest paid rate prevailing date deposit tenure deposit withdrawn amount remained bank contracted rate  whichever lower  deducting applicable penal charge premature withdrawal per Terms & Conditions Fixed Deposit Accounts Bank  2. **Penalty Charge Specification **  penal charge premature closure Fixed Deposits including partial closure fixed bank Fixed Deposits booked   renewed 20th May  2022.  3. **Penalty Table **  *  Tenure Fixed Deposit Penalty   *  Less equal 180 days Nil   *  Greater 180 days less equal 364 days 0.50%   *  Greater equal 365 days 1.00%  4. **TDS Rates **  *  Tax Rate Health Education Cess TOTAL   *  Resident Individuals & HUF 10% Nil 10%   *  Corporate Entity 10% Nil 10%   *  NRO   foreign companies 30% 4% 31.2%   *  Firms 10% Nil 10%  5. **TDS Disclaimer **  per section 206AA introduced 

In [58]:
cursor.execute(f"drop table test_table3;")

In [33]:
check_table_exists(table_name)

False

In [21]:
close_db()

NameError: name 'close_db' is not defined

In [22]:
cursor.close()
connection.close()

In [12]:
from langchain.schema import Document
text_splitter = CharacterTextSplitter(chunk_size=900, chunk_overlap=400)
splits = text_splitter.split_documents([Document(page_content="Hello world"*int(10e7))])

In [7]:
[i for i in range(20,10,2)]

[]

In [6]:
range(7,2,-1)

range(7, 2, -1)

In [11]:
from supabase import create_client
import os
spb = create_client(os.getenv("SUPABASE_URL"), os.getenv("SUPABASE_KEY"))

In [16]:
res = spb.table("HDFC").select("tenure_end,rate").execute()

In [25]:
data = res.data
for row in data:
    row["number_of_days_in_tenure"] = row.pop("tenure_end")
    row["interest_rate"] = row.pop("rate")

In [26]:
data

[{'number_of_days_in_tenure': 14, 'interest_rate': 3.5},
 {'number_of_days_in_tenure': 29, 'interest_rate': 3.5},
 {'number_of_days_in_tenure': 45, 'interest_rate': 4},
 {'number_of_days_in_tenure': 60, 'interest_rate': 5},
 {'number_of_days_in_tenure': 89, 'interest_rate': 5},
 {'number_of_days_in_tenure': 180, 'interest_rate': 5},
 {'number_of_days_in_tenure': 270, 'interest_rate': 6.25},
 {'number_of_days_in_tenure': 365, 'interest_rate': 6.5},
 {'number_of_days_in_tenure': 455, 'interest_rate': 7.1},
 {'number_of_days_in_tenure': 540, 'interest_rate': 7.55},
 {'number_of_days_in_tenure': 630, 'interest_rate': 7.55},
 {'number_of_days_in_tenure': 730, 'interest_rate': 7.2},
 {'number_of_days_in_tenure': 880, 'interest_rate': 7.4},
 {'number_of_days_in_tenure': 1095, 'interest_rate': 7.4},
 {'number_of_days_in_tenure': 1550, 'interest_rate': 7.4},
 {'number_of_days_in_tenure': 1825, 'interest_rate': 7.25},
 {'number_of_days_in_tenure': 1650, 'interest_rate': 7.25},
 {'number_of_days_

In [9]:
import requests
import json
import os 
from dotenv import load_dotenv

load_dotenv()
res = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {os.getenv("OPENROUTER_API_KEY")}",
            "Content-Type": "application/json"
        },
        data=json.dumps({
            "model": "qwen/qwen2.5-vl-3b-instruct:free",
            "messages": [
            {
                "role": "user",
                "content": [
                {
                    "type": "text",
                    "text": """
                            hI HOW ARE YOU.
                            """
                }                
                ]
                }
                ],
            
            })
        )

"Hello! I'm here to help you with any questions or tasks you might have. Is there something specific you would like to know or discuss?"

In [1]:
from LLM_query_resolver import QueryResponder
db = QueryResponder(model_name="qwen/qwen3-32b:free")

In [2]:
ans = db.resolve_query("what is are the interest rate for hdfc if i want to invest for 1 and a half year")
print(ans)


                                        HDFC Fixed deposite rates are:
                                        --------------------------
                                        [{'tenure_start': 7, 'number_of_days_in_tenure': 14, 'interest_rate': 3.5}, {'tenure_start': 15, 'number_of_days_in_tenure': 29, 'interest_rate': 3.5}, {'tenure_start': 30, 'number_of_days_in_tenure': 45, 'interest_rate': 4}, {'tenure_start': 46, 'number_of_days_in_tenure': 60, 'interest_rate': 5}, {'tenure_start': 61, 'number_of_days_in_tenure': 89, 'interest_rate': 5}, {'tenure_start': 90, 'number_of_days_in_tenure': 180, 'interest_rate': 5}, {'tenure_start': 181, 'number_of_days_in_tenure': 270, 'interest_rate': 6.25}, {'tenure_start': 271, 'number_of_days_in_tenure': 365, 'interest_rate': 6.5}, {'tenure_start': 365, 'number_of_days_in_tenure': 455, 'interest_rate': 7.1}, {'tenure_start': 456, 'number_of_days_in_tenure': 540, 'interest_rate': 7.55}, {'tenure_start': 541, 'number_of_days_in_tenure': 630, 'in

In [5]:
querry = "comapre hdfc and icicic"

In [6]:
db.query_relevant_data("hdfc",querry)

[(1,
  'Based on the provided input text  here are the exact sentences or clauses identified as part of the Terms and Conditions  preserved verbatim and without alteration  1. **Terms & Conditions document lays Terms Conditions shall applicable accounts existing may opened anytime future HDFC Bank.** 2. **words  I  me  Customer refer person open account shall include singular plural. Reference masculine shall include feminine neuter gender. Bank refers HDFC Bank Ltd.  banking company incorporated India Companies Act 1956 registered office HDFC Bank House  Senapati Bapat Marg  Lower Parel  Mumbai-400013.** 3. **1.1 agree abide Bank Terms Conditions rules force changes thereto Terms Conditions time time relating account communicated made available Bank website.** 4. **1.2 agree opening maintenance account subject rules regulations introduced amended time time Reserve Bank India.** 5. **1.3 agree bank opening deposit account  carry due diligence required Know Customer guidelines bank. wou

In [24]:
bank = "HDFC"
terms = "\\n".join([chunk[1] for chunk in db.query_relevant_data(bank,f"{querry} regarding the bank {bank}")])
terms

'Based on the provided input text  here are the exact sentences or clauses identified as part of the Terms and Conditions  preserved verbatim and without alteration  1. **Terms & Conditions document lays Terms Conditions shall applicable accounts existing may opened anytime future HDFC Bank.** 2. **words  I  me  Customer refer person open account shall include singular plural. Reference masculine shall include feminine neuter gender. Bank refers HDFC Bank Ltd.  banking company incorporated India Companies Act 1956 registered office HDFC Bank House  Senapati Bapat Marg  Lower Parel  Mumbai-400013.** 3. **1.1 agree abide Bank Terms Conditions rules force changes thereto Terms Conditions time time relating account communicated made available Bank website.** 4. **1.2 agree opening maintenance account subject rules regulations introduced amended time time Reserve Bank India.** 5. **1.3 agree bank opening deposit account  carry due diligence required Know Customer guidelines bank. would requ

In [ ]:

print(db._context_fetcher(querry))


                                        HDFC Fixed deposite rates are:
                                        --------------------------
                                        [{'tenure_start': 7, 'tenure_end': 14, 'rate': 3.5}, {'tenure_start': 15, 'tenure_end': 29, 'rate': 3.5}, {'tenure_start': 30, 'tenure_end': 45, 'rate': 4}, {'tenure_start': 46, 'tenure_end': 60, 'rate': 5}, {'tenure_start': 61, 'tenure_end': 89, 'rate': 5}, {'tenure_start': 90, 'tenure_end': 180, 'rate': 5}, {'tenure_start': 181, 'tenure_end': 270, 'rate': 6.25}, {'tenure_start': 271, 'tenure_end': 365, 'rate': 6.5}, {'tenure_start': 365, 'tenure_end': 455, 'rate': 7.1}, {'tenure_start': 456, 'tenure_end': 540, 'rate': 7.55}, {'tenure_start': 541, 'tenure_end': 630, 'rate': 7.55}, {'tenure_start': 631, 'tenure_end': 730, 'rate': 7.2}, {'tenure_start': 731, 'tenure_end': 880, 'rate': 7.4}, {'tenure_start': 881, 'tenure_end': 1095, 'rate': 7.4}, {'tenure_start': 1096, 'tenure_end': 1550, 'rate': 7.4}, {'tenure_

Based on the provided HDFC Fixed Deposit rates, for a 1-year investment, the interest rate is 3.5% for tenures between 7 days and 14 days, but since you're looking for a 1-year investment, the interest rate would be 4% for a tenure of 30 days to 45 days, 5% for a tenure of 46 days to 60 days, and so on.

However, since you're comparing HDFC with ICICI, I would suggest checking the interest rates of both banks for a 1-year fixed deposit. As of my last update, ICICI Bank's 1-year fixed deposit interest rates might be slightly different and could be more favorable, but I recommend checking the latest rates from both banks' official websites or visiting a branch for the most accurate and up-to-date information.

That being said, here's a general comparison based on the provided HDFC rates:

- HDFC: For a 1-year investment, the interest rate could be around 5% or slightly higher, depending on the exact tenure and any potential upgrades in interest rates.

To give you a better answer, I woul

In [5]:
True if "HDFC" in " COMAPRE HDFC AND ICICIC" else False

True

In [20]:
terms = [chunk[1] for chunk in query_relevant_data("HDFC",querry)]
f"""
                 terms And Conditions:
                ---------------------------
                {"\n".join(terms)}
                ---------------------------
                """

'\n                 terms And Conditions:\n                ---------------------------\n                Based on the provided input text  here are the exact sentences or clauses identified as part of the Terms and Conditions  preserved verbatim and without alteration  1. **Terms & Conditions document lays Terms Conditions shall applicable accounts existing may opened anytime future HDFC Bank.** 2. **words  I  me  Customer refer person open account shall include singular plural. Reference masculine shall include feminine neuter gender. Bank refers HDFC Bank Ltd.  banking company incorporated India Companies Act 1956 registered office HDFC Bank House  Senapati Bapat Marg  Lower Parel  Mumbai-400013.** 3. **1.1 agree abide Bank Terms Conditions rules force changes thereto Terms Conditions time time relating account communicated made available Bank website.** 4. **1.2 agree opening maintenance account subject rules regulations introduced amended time time Reserve Bank India.** 5. **1.3 agr

In [37]:
obj = response.json()

In [38]:
print(obj["choices"][0]["message"]["content"])


The President of Ukraine is Volodymyr Zelenskyy.
